In [2]:
import csv
txt = 'train.txt'
with open(txt, 'r') as file:
    data = file.readlines()
validation_len = len(data)/5
validation_set = data[0:validation_len]
train_set = data[validation_len:]

In [3]:
# Training and validation
precision_avg = 0
recall_avg = 0
F_measure_avg = 0
for i in range(1,11):
    # cross validation
    validation_len = len(data)/10
    validation_set = data[(i-1)*validation_len:i*validation_len]
    train_set = data[0:(i-1)*validation_len]
    train_set.extend(data[i*validation_len:len(data)])
    
    
    # build a lexicon that contains all the entities in the training set
    word_NE = {}
    for i in range(0,len(train_set),3):
        word_line = train_set[i].replace('\r\n','').split('\t')
        NE_line = train_set[i+2].replace('\r\n','').split('\t')


        NE = []
        begin = False
        for j in range(0,len(word_line)):
            word = word_line[j]
            ne = NE_line[j]

            if j==0 and ne[0]=='B':
                NE.append(word)
                begin = True
            if j>0 and ne[0]=='I' and begin == True:
                NE.append(word)
                if j==len(word_line)-1:
                    word_NE[tuple(NE)]=ne[2:]
                    NE = []
            if j>0 and ne[0]!='I' and begin == True:
                word_NE[tuple(NE)]=NE_line[j-1][2:]
                NE = []
                begin = False
            if j>0 and ne[0]=='B':
                NE.append(word)
                begin = True
                if j==len(word_line)-1:
                    word_NE[tuple(NE)]=NE_line[j][2:]
                    NE = []
                    
    word_NE_prefix = {}
    for t in word_NE.keys():
        if t[0] not in word_NE_prefix:
            word_NE_prefix[t[0]]=[t[0:]]
        else:
            word_NE_prefix[t[0]].append(t[0:])
    
    # identify only those named entities that are part of the lexicon above 
    correct = 0
    all_correct = 0
    predicted = 0
    for i in range(0,len(validation_set),3):
        word_line = validation_set[i].replace('\r\n','').split('\t')
        NE_line = validation_set[i+2].replace('\r\n','').split('\t')

        for j in range(0,len(word_line)):
            word = word_line[j]
            ne = NE_line[j]
            
            if ne[0] == 'B':
                all_correct +=1
        
        end = 0
        for j in range(0,len(word_line)):
            word = word_line[j]
            ne = NE_line[j]    
            
            if word in word_NE_prefix and j>=end:
                match_list = word_NE_prefix[word]
                for t in match_list:
                    end_index = min(j+len(t),len(word_line))
                    if tuple(word_line[j:end_index]) == t:
                        predicted +=1
                        tag = word_NE[t]
                        end = end_index
                        if tag == ne[2:]:
                            correct +=1
                        break
        
                        
                
            
            
            
    precision = correct/float(predicted)
    recall  = correct/float(all_correct)
    F_measure = 2*precision*recall/(precision + recall)
    print "precision: "+str(precision), "recall: "+str(recall),"F: "+str(F_measure)
    
    precision_avg += precision
    recall_avg += recall
    F_measure_avg += F_measure

print "avg_precision: "+str(precision_avg/10.0),"recall: "+str(recall_avg/10.0),"F: "+str(F_measure_avg/10.0)  

precision: 0.714146341463 recall: 0.644366197183 F: 0.677464136974
precision: 0.788229111929 recall: 0.653025685677 F: 0.714285714286
precision: 0.842325581395 recall: 0.709639498433 F: 0.770310506168
precision: 0.720283257461 recall: 0.656221198157 F: 0.686761514348
precision: 0.86102062975 recall: 0.651334702259 F: 0.741641337386
precision: 0.738916256158 recall: 0.617848970252 F: 0.67298105683
precision: 0.790966386555 recall: 0.652230402772 F: 0.714929978638
precision: 0.761761761762 recall: 0.667836770513 F: 0.711713818097
precision: 0.803813346713 recall: 0.682573498083 F: 0.738248847926
precision: 0.806574559314 recall: 0.684593610999 F: 0.740594925634
avg_precision: 0.78280372325 recall: 0.661967053433 F: 0.716893183629


In [ ]:
# helper function to output the NE results
def tagging(begin_index,end_index,tag):
    global PER,LOC,ORG,MISC
    if tag == 'PER':
         PER =  PER+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'LOC':
        LOC =  LOC+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'ORG':
        ORG =  ORG+str(begin_index)+'-'+str(end_index)+' '
    if tag == 'MISC':
        MISC = MISC+str(begin_index)+'-'+str(end_index)+' '

In [51]:
# predict the test data and output the result

headers = ['Type','Prediction']
rows = []
PER = ''
LOC = ''
ORG = ''
MISC = ''

txt = 'train.txt'
with open(txt, 'r') as file:
    train_set = file.readlines()

word_NE = {}
for i in range(0,len(train_set),3):
    word_line = train_set[i].replace('\r\n','').split('\t')
    NE_line = train_set[i+2].replace('\r\n','').split('\t')


    NE = []
    begin = False
    for j in range(0,len(word_line)):
        word = word_line[j]
        ne = NE_line[j]

        if j==0 and ne[0]=='B':
            NE.append(word)
            begin = True
        if j>0 and ne[0]=='I' and begin == True:
            NE.append(word)
            if j==len(word_line)-1:
                word_NE[tuple(NE)]=ne[2:]
                NE = []
        if j>0 and ne[0]!='I' and begin == True:
            word_NE[tuple(NE)]=NE_line[j-1][2:]
            NE = []
            begin = False
        if j>0 and ne[0]=='B':
            NE.append(word)
            begin = True
            if j==len(word_line)-1:
                word_NE[tuple(NE)]=NE_line[j][2:]
                NE = []

word_NE_prefix = {}
for t in word_NE.keys():
    if t[0] not in word_NE_prefix:
        word_NE_prefix[t[0]]=[t[0:]]
    else:
        word_NE_prefix[t[0]].append(t[0:])
            
txt = 'test.txt'
with open(txt, 'r') as file:
    data = file.readlines()

for i in range(0,len(data),3):
        word_line = data[i].replace('\r\n','').split('\t')
        num_line = data[i+2].replace('\r\n','').split()

        
        end = 0
        for j in range(0,len(word_line)):
            word = word_line[j]
            
            if word in word_NE_prefix and j>=end:
                match_list = word_NE_prefix[word]
                for t in match_list:
                    end_index = min(j+len(t),len(word_line))
                    if tuple(word_line[j:end_index]) == t:
                        end = end_index
                        tag = word_NE[t]  
                        tagging(num_line[j],num_line[end_index-1],tag)
                        break

rows.append(('PER',PER))
rows.append(('LOC',LOC))
rows.append(('ORG',ORG))
rows.append(('MISC',MISC))
               
            
with open('baseline.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)  